<a href="https://colab.research.google.com/github/inesbsilveira/estufas/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [8]:
import datetime
import folium

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def gee_images(extent, start_date, end_date, band_name, folder_name):
    lonmin, lonmax, latmin, latmax = extent

    region = ee.Geometry.Polygon(
        [[
            [lonmin, latmin],
            [lonmax, latmin],
            [lonmax, latmax],
            [lonmin, latmax]
        ]])

    # Create an image collection for Sentinel-2A
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region)

    # Select the band using the band_name parameter
    band = collection.select(band_name)

    # Reduce the collection to a single image using median (or other reducers)
    median_image = band.median()

    # Create a region of interest mask
    roi_image = ee.Image().paint(region, 1, 1)

    # Mask the median image with the ROI
    band_masked = median_image.updateMask(roi_image)

    # Print the resulting image (for debugging purposes)
    print("Band image:", band_masked)

    task = ee.batch.Export.image.toDrive(
        image=band_masked,  # Corrected the variable name
        description=f'{band_name}_Image',  # Used f-string for dynamic description
        folder=folder_name,
        scale=10,
        maxPixels=1e13
    )

    # Start the export task
    task.start()


In [ ]:
# Example usage
gee_images((-8.796344, -8.741399, 37.440509, 37.559064), '2022-07-01', '2022-07-31', 'B2', 'Estufas')

In [ ]:
# Define the ROI polygon
lonmin = -8.796344
lonmax = -8.741399
latmin = 37.440509
latmax = 37.559064

regiao = ee.Geometry.Polygon(
    [[[lonmin, latmin],
      [lonmax, latmin],
      [lonmax, latmax],
      [lonmin, latmax]]])

# Define the date range
start_date = '2022-07-01'
end_date = '2022-07-31'

# Create an image collection for Sentinel-2A
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(regiao)

# Select the B2 (blue) band
blue_band = collection.select('B2')

# Reduce the collection to a single image using median (or other reducers)
median_image = blue_band.median()

# Create a region of interest mask
roi_image = ee.Image().paint(regiao, 1, 1)

# Mask the median image with the ROI
blue_band_masked = median_image.updateMask(roi_image)

# Print the resulting image
print("Blue band image:", blue_band_masked)

task = ee.batch.Export.image.toDrive(image=blue_band_masked,
                                    description='B2_Image_t',
                                    folder='Estufas',
                                    scale=10,
                                    maxPixels=1e13)

# Start the export task
task.start()
